# 네이버 뉴스 데이터 가져오기

> 1. 네이버 오픈 API로 기사 URL 크롤링
> 2. bs4 + selenium으로 기사 제목 및 내용 크롤링

In [15]:
# 검색 API 사용예제 https://developers.naver.com/docs/search/blog/

import os
import sys
import urllib.request
import requests

news_data = []

client_id = "jeNMpofTMshM4WUgct_o"
client_secret = "zB7DowAiIL" # 발급받은 client_secret
encText = urllib.parse.quote("빅데이터") # encText : 실제 url에 "파이썬" 단어를 붙여주는 코드
# id와 secret을 가지고 오픈api를 사용한다는 것은 
# 네이버 서버에 ~~를 검색할 것이라고 요청을 보내는 것
# id와 secret를 발급받은 이유는 코드 상에서 돌리는 내용들이 네이버가 검증을 실제로 할 수 있음
# query를 날릴 때(request 보낼 때) id와 secret 동시에 보내줘야해

url = "https://openapi.naver.com/v1/search/news?query=" + encText # json 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
# blog가 아닌 news기사를 가져오고싶기때문에 news라고 바꿔줌
# query= + encText : 실제로 "파이썬"단어를 검색한 것과 똑같은 url 만들어줌


# urllib.request 라이브러리는 말그대로 네이버 서버에 ~내용 좀 줘 요청하는 것
# 이 라이브러리 필요한 것: 어디에 접속할 건지 접속 url줘야하고, url의 정보를 줘야해
# request보낼 때 나는 openapi를 받은 사람이야라고 인증을 위한 키를 들고가야겠지
# 어디에 주냐 -> requset의 header 정보에 client id와 secret을 들고 request를 보내면 ok해주는 코드
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id", client_id)
request.add_header("X-Naver-Client-Secret", client_secret)
# urlopen: request를 보냈을 때 url이 연결되면, 연결을 열어줌(접속이 됐다라는 뜻)
# 그럼 이제 데이터를 받아올 수 있지
response = urllib.request.urlopen(request)
# getcode(): 첫번째로 code를 받아와
rescode = response.getcode()


# 실제로 네트워크같은 것은 예를 들어, 노트북에서 request보내면 wifi의 무선 랜카드를 통해서 실제로 정보를 물리적으로 전파를 통해 보내줌
# 그럼 wifi 단말기로 전달되고, 외부 네트워크로 뻗어나감. 그 뻗어나간 외부 네트워크를 통해 네이버의 서버로 가
# 만약 네트워크가 가는 도중에 길을 잃었어. 다른 패키지와 충돌이 나는 등 또는 인증을 못한 경우(오타, 사용량 제한)
# request가 안됐다는 것을 어떻게 아냐 -> response code 정보로 줌(너 request 도착안했어, 왔는데 ~가 잘못됐어, 너 일일사용량 넘었어 등등)
# 홈페이지에 error 코드 숫자 나와있음


if(rescode==200): # 성공
#    강사님이 따로 수정한 코드
#    response_body = response.read() # 기존 코드
    result = requests.get(response.geturl(),
                          headers={"X-Naver-Client-Id":client_id,
                                   "X-Naver-Client-Secret":client_secret}
                         )
    # requests.get(): url주소 던져주면 내용을 받아오는 함수
    
    news_data.append(result.json())
    # json(): javascript object notation의 약자
    # 파이썬 입장에서 딕셔너리처럼 key,value의 형태로 저장됨
    # 즉 결과를 딕셔너리 형태로 저장해줌
    # result.json(): 통째로 하나로 가져옴
    
#    print(response_body.decode('utf-8')) # 기존 코드
else: # 실패
    print("Error Code:" + rescode)
    

    
# 검색 api는 본문을 제공해주지 않아
# decription이라고 되어있는 속성에다가 전체 뉴스기사의 요약본(요약알고리즘 쓴 것 같음)을 가져옴
# decription을 key sentence라고 봐도되는데, 읽어보니까 그건 아닌 것 같음..
# 중요한 건 어쨌든 본문 내용 가져올 수 없고, 가져오려면 따로 뉴스기사에 접속해서 가져와야해
# 그래서 openapi를 통해 주소만 가져와서 쓸 거야
# 즉 데이터를 사용하기 위해 결과를 저장하는 코드로 변경함

### 네이버 OPEN API를 통해 가져온 데이터 확인하기.

In [16]:
news_data
# 우리가 필요한 건 링크가 필요해
# 어떻게 갖고오냐가 문젠데, json으로 가져왔는데 리스트 원소가 지금 하나야
# 그래서 news_data[0]을 하면 큰 리스트가 벗겨 짐 -> 10개가 한꺼번에 저장됨

[{'lastBuildDate': 'Mon, 01 Jun 2020 19:33:55 +0900',
  'total': 732559,
  'start': 1,
  'display': 10,
  'items': [{'title': '[PRNewswire] Shanghai Electric, SEunicloud 업그레이드 통해 산업 디지털화...',
    'originallink': 'http://yna.kr/AKR20200601167600009?did=1195m',
    'link': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=001&aid=0011648367',
    'description': "이 전자상거래 플랫폼은 <b>빅 데이터</b> 분석을 통해 지능형 맞춤 제품과 서비스를 위한 통합적인 데이터 중심 솔루션 양성이라는 희망을 키운다. Shanghai Electric 그룹 이사 및 사장 Huang Ou는 &quot;자사는 '새로운... ",
    'pubDate': 'Mon, 01 Jun 2020 19:30:00 +0900'},
   {'title': '경북도 인재개발원, 6월 1일부터 공무원교육 본격 재개',
    'originallink': 'http://www.ccreview.co.kr/news/articleView.html?idxno=262861',
    'link': 'http://www.ccreview.co.kr/news/articleView.html?idxno=262861',
    'description': '한편, 도 인재개발원은 금년도 「변화를 선도하는 창의적 인재양성」을 목표로 ▲창의적‧실용적 교육 ▲개방형 교육을 통한 도정시책 공유 ▲새로운 트렌드를 반영하는 스마트 교육 ▲<b>빅데이터</b> 활용능력 확보... ',
    'pubDate': 'Mon, 01 Jun 2020 19:28:00 +0900'},
   {'title': "인천대 제3대 총장

In [17]:
news_data[0]
# items라는 속성안에 리스트로 쭉 있음

{'lastBuildDate': 'Mon, 01 Jun 2020 19:33:55 +0900',
 'total': 732559,
 'start': 1,
 'display': 10,
 'items': [{'title': '[PRNewswire] Shanghai Electric, SEunicloud 업그레이드 통해 산업 디지털화...',
   'originallink': 'http://yna.kr/AKR20200601167600009?did=1195m',
   'link': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=001&aid=0011648367',
   'description': "이 전자상거래 플랫폼은 <b>빅 데이터</b> 분석을 통해 지능형 맞춤 제품과 서비스를 위한 통합적인 데이터 중심 솔루션 양성이라는 희망을 키운다. Shanghai Electric 그룹 이사 및 사장 Huang Ou는 &quot;자사는 '새로운... ",
   'pubDate': 'Mon, 01 Jun 2020 19:30:00 +0900'},
  {'title': '경북도 인재개발원, 6월 1일부터 공무원교육 본격 재개',
   'originallink': 'http://www.ccreview.co.kr/news/articleView.html?idxno=262861',
   'link': 'http://www.ccreview.co.kr/news/articleView.html?idxno=262861',
   'description': '한편, 도 인재개발원은 금년도 「변화를 선도하는 창의적 인재양성」을 목표로 ▲창의적‧실용적 교육 ▲개방형 교육을 통한 도정시책 공유 ▲새로운 트렌드를 반영하는 스마트 교육 ▲<b>빅데이터</b> 활용능력 확보... ',
   'pubDate': 'Mon, 01 Jun 2020 19:28:00 +0900'},
  {'title': "인천대 제3대 총장 최종 후보에 '이찬근 무역

In [26]:
news_data[0]['items'] # 딕셔너리의 인덱싱 방법이지

[{'title': '[PRNewswire] Shanghai Electric, SEunicloud 업그레이드 통해 산업 디지털화...',
  'originallink': 'http://yna.kr/AKR20200601167600009?did=1195m',
  'link': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=001&aid=0011648367',
  'description': "이 전자상거래 플랫폼은 <b>빅 데이터</b> 분석을 통해 지능형 맞춤 제품과 서비스를 위한 통합적인 데이터 중심 솔루션 양성이라는 희망을 키운다. Shanghai Electric 그룹 이사 및 사장 Huang Ou는 &quot;자사는 '새로운... ",
  'pubDate': 'Mon, 01 Jun 2020 19:30:00 +0900'},
 {'title': '경북도 인재개발원, 6월 1일부터 공무원교육 본격 재개',
  'originallink': 'http://www.ccreview.co.kr/news/articleView.html?idxno=262861',
  'link': 'http://www.ccreview.co.kr/news/articleView.html?idxno=262861',
  'description': '한편, 도 인재개발원은 금년도 「변화를 선도하는 창의적 인재양성」을 목표로 ▲창의적‧실용적 교육 ▲개방형 교육을 통한 도정시책 공유 ▲새로운 트렌드를 반영하는 스마트 교육 ▲<b>빅데이터</b> 활용능력 확보... ',
  'pubDate': 'Mon, 01 Jun 2020 19:28:00 +0900'},
 {'title': "인천대 제3대 총장 최종 후보에 '이찬근 무역학부 교수'",
  'originallink': 'http://www.kyeonggi.com/news/articleView.html?idxno=2291771',
  'link': 'http://www.kyeonggi

In [19]:
print(news_data[0]['items'][0]['link'])
# items도 리스트야
# 이중에서 link를 가지고와야하니까 [0] 첫번째 원소에 ['link']

# Tip: 파이썬에서 print()를 통해 url찍으면 바로 접속가능함

# 네이버 기사 웹크롤링 할 때 가장 중요한 점
# 네이버 기사 종류가 너무 많아서 기사를 하나하나 전부 가져오기가 굉장히 힘들어
# 기사마다 tag 이름/종류가 다 달라서 무조건 다 가져와야겠다하면 하나하나 코드로 다 가져와야함
# 실제론 여러 트릭을 쓰는데, 보통 쓰는게 모바일 접속환경으로 들어가 (m.news.~~)
# 모바일 규격을 일정하게 만들어서 publish해줘야해서 규격을 맞추는 작업때문에 네이버 뉴스 템플릿으로 많이 바뀌어서 보임
# 일단 네이버 뉴스만 가져옴(url로 따지면 news.naver.com만) (tag 동일하게 가져오기위해)

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=001&aid=0011648367


### 가져온 URL이 네이버 뉴스인지 확인하기.

In [27]:
page_news_link = []

for item in news_data[0]['items']:
    link = item['link']
    if 'naver' in link :
        page_news_link.append(link)
            
len(page_news_link)

1

### 네이버 뉴스 기사 크롤링

In [ ]:
# url을 위처럼 10페이지가 아닌 여러 페이지에서 가져올 수 있는 방법

In [29]:
news_data = []
page_count = 3 # 몇 페이지 가져올 것인지

client_id = "jeNMpofTMshM4WUgct_o"
client_secret = "zB7DowAiIL" # 발급받은 client_secret
encText = urllib.parse.quote("빅데이터")

for idx in range(page_count):
    # 10씩 더해주는데 1에다가 더해줘, inx는 0부터 시작하지
    # => idx * 10 + 1
    
    url = "https://openapi.naver.com/v1/search/news?query=" + encText + "&start=" + str(idx * 10 + 1) # json 결과
    # "&start=" : 규칙, 알고있자
    
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    
    # page별로 관리하고싶으면 2차원리스트 만들어서 page당 링크를 저장하면 됨
    # 굳이 안 그러고 싶으면 통째로 저장해도됨

    if(rescode==200): # 성공
    #    response_body = response.read() # 기존 코드
        result = requests.get(response.geturl(),
                              headers={"X-Naver-Client-Id":client_id,
                                       "X-Naver-Client-Secret":client_secret}
                             )

        news_data.append(result.json())

    #    print(response_body.decode('utf-8')) # 기존 코드
    else: # 실패
        print("Error Code:" + rescode)

In [33]:
print(news_data[0]['items'][0]['link'])
print(len(news_data))

https://platum.kr/archives/142346
3


In [34]:
# page별로 관리하는 코드로 바꾸자
naver_news_link = []

# 아까는 뉴스 데이터가 1개 리스트, 지금은 아니야 (3개)
for page in news_data: # page : news_data의 [0]번째부터 가져온다고 생각하면 돼
    
    page_news_link = []

    for item in page['items']:
        link = item['link']
        if 'naver' in link:
            page_news_link.append(link)

    naver_news_link.append(page_news_link)
    
print(len(naver_news_link))

3


In [45]:
naver_news_link = []

for page in news_data:
    
    page_news_link = []

    for item in page['items']:
        link = item['link']
        if 'naver' in link:
            page_news_link.append(link)

    naver_news_link.append(page_news_link)
    
print(len(naver_news_link))

for page in naver_news_link:
    for link in page:
        print(link)

3
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=629&aid=0000027777
http://www.segyebiz.com/newsView/20200601516070?OutUrl=naver
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=001&aid=0011648367
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=022&aid=0003470640
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=082&aid=0001006400
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=082&aid=0001006395
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=417&aid=0000549680
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=011&aid=0003747027
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=055&aid=0000818481
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=014&aid=0004435246


In [28]:
import pandas as pd
import numpy as np
from selenium import webdriver
from tqdm import tqdm_notebook
import requests
import pickle
import re
import ast

from bs4 import BeautifulSoup 
from urllib.request import urlopen
import urllib
import time

In [47]:
# 가상 크롬드라이버를 불러옴.
# 윈도우 10의 경우 chromedriver.exe
driver = webdriver.Chrome("./driver/chromedriver")

In [51]:
# naver_news_link에 있는 url 하나하나에 접속해서 타이틀과 본문 가져오자

naver_news_title = []
naver_news_content = []

for n in tqdm_notebook(range(len(naver_news_link))):
    #print(n)
    news_page_title = []
    news_page_content = []
    
    for idx in tqdm_notebook(range(len(naver_news_link[n]))):
          
    ########### 긁어온 URL로 접속하기 ############    
        try:
            driver.get(naver_news_link[n][idx])
            print(naver_news_link[n][idx])
            
        except:
            print("Timeout!")
            continue
            
# try,except쓴 이유는 실제로 크롤링할 때 에러가 많이 뜸
# 네트워크 때문에, 실제 기사가 있었다가 없어진 이유 등등        
        
        try:
            response = driver.page_source
            
        except UnexpectedAlertPresentException:
            driver.switch_to_alert().accept()
            print("게시글이 삭제된 경우입니다.")
            continue
        
        soup = BeautifulSoup(response, "html.parser")
        
        ###### 뉴스 타이틀 긁어오기 ######
        
        title = None
        
        try:
            item = soup.find('div', class_="article_info")
            title = item.find('h3', class_="tts_head").get_text()
            #print(title)

        except:
            title = "OUTLINK"
        
        #print(title)
        news_page_title.append(title)
        
        
        ###### 뉴스 본문 긁어오기 ######
 # 본문 가져오는 방법 되게 다양함. 강사님은 이 코드가 어려운데 가장 잘 된대
        
        doc = None
        text = ""
                
        data = soup.find_all("div", {"class" : "_article_body_contents"})
         # class가 _article_body_contents인 것을 find_all -> 모두 다 가져온 다음에
        # 그중에서 text만 가져오는 것
        
        if data:
            for item in data:

                text = text + str(item.find_all(text=True)).strip()
                text = ast.literal_eval(text)
                doc = ' '.join(text)
   
        else:
            doc = "OUTLINK"
            
        news_page_content.append(doc.replace('\n', ' '))

                
    naver_news_title.append(news_page_title)
    naver_news_content.append(news_page_content)

    time.sleep(2) # 이거 안해주면 디도스?같은 걸로 인식해서 막아버린대
    
    
print(naver_news_title[0])
print("==================================")
print(naver_news_content[0])

C:\Users\hyun\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


C:\Users\hyun\lib\site-packages\ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=629&aid=0000027777
http://www.segyebiz.com/newsView/20200601516070?OutUrl=naver
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=001&aid=0011648367



https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=022&aid=0003470640
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=082&aid=0001006400



https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=082&aid=0001006395
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=417&aid=0000549680
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=011&aid=0003747027
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=055&aid=0000818481
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=014&aid=0004435246


["'76조 투입' 한국판 뉴딜에 어떤 내용 담겼나", 'OUTLINK', '[PRNewswire] Shanghai Electric, SEunicloud 업그레이드 통해 산업 디지털화 가속']
["   본문 내용     TV플레이어     // TV플레이어     // flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}    정부가 '디지털 뉴딜'과 '그린 뉴딜' 등으로 나눈 한국판 뉴딜에 오는 2025년까지 총 76조 원을 투입한다고 1일 밝혔다. 사진은 홍남기 경제부총리 겸 기획재정부 장관. /이동률 기자 정부, 한국판 뉴딜 '디지털·그린'에 76조 원 투입 [더팩트ㅣ이성락 기자] '포스트 코로나' 시대를 대비하고 장기적으로 한국 경제의 도약을 준비하는 프로젝트인 '한국판 뉴딜'에 오는 2025년까지 총 76조 원이 투입된다. 정부가 1일 발표한 한국판 뉴딜은 '디지털 뉴딜'과 '그린 뉴딜'을 양대 축으로, '고용안전망 강화' 토대 위에서 추진된다. 이날 정부는 기획재정부 등 관계 부처 합동으로 발표한 '2020년 하반기 경제정책방향'(하경정)

### 크롤링 결과 확인하기.

In [52]:
print(naver_news_title[0])

["'76조 투입' 한국판 뉴딜에 어떤 내용 담겼나", 'OUTLINK', '[PRNewswire] Shanghai Electric, SEunicloud 업그레이드 통해 산업 디지털화 가속']


In [53]:
print(naver_news_content[0])

["   본문 내용     TV플레이어     // TV플레이어     // flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}    정부가 '디지털 뉴딜'과 '그린 뉴딜' 등으로 나눈 한국판 뉴딜에 오는 2025년까지 총 76조 원을 투입한다고 1일 밝혔다. 사진은 홍남기 경제부총리 겸 기획재정부 장관. /이동률 기자 정부, 한국판 뉴딜 '디지털·그린'에 76조 원 투입 [더팩트ㅣ이성락 기자] '포스트 코로나' 시대를 대비하고 장기적으로 한국 경제의 도약을 준비하는 프로젝트인 '한국판 뉴딜'에 오는 2025년까지 총 76조 원이 투입된다. 정부가 1일 발표한 한국판 뉴딜은 '디지털 뉴딜'과 '그린 뉴딜'을 양대 축으로, '고용안전망 강화' 토대 위에서 추진된다. 이날 정부는 기획재정부 등 관계 부처 합동으로 발표한 '2020년 하반기 경제정책방향'(하경정)을 통해 한국판 뉴딜 추진 방침을 발표했다. 정부는 한국판 뉴딜 추진에 문재인 정부 임기 말인 2022년까지 31조3000억 원을 투자해 55만 개 수준의 일자리를 창출한다는 계획이다. 또한, 한국판 뉴딜 완성을 위해 문재인 정부 이후인 2023년부터 2025년까지 45조 원 추가 투입해 국가 산업 재편과 디지털 강국의 입지를 공고히 하겠다는 방침이다. 구체적으로 디지털 뉴딜 중점 과제는 △ D.N.A.  생태계 강화 △디지털 포용 및 안전망 구축 △비대면 산업 육성 △사회간접자본( SOC ) 디지털화 등이다. 우선 정부는 2020년까지 6조4000억 원을 투입해  D.N.A.  생태계 강화에 나선다. 데이터( Data )와 네트워크( Network ), 인공지능( AI ) 사업을 키워 약 22만2000개의 일자리를 창출한다는 계획이다. 정부는 금융·환경·문화·교통·헬스케어 등 국민생활과 밀접한 15개 분야 빅데이터 플랫폼을 구축하고, 14만 개 공공데이터를 개방하기로 했다. 또  AI  학습용 데이터 700종을 추가 구축하기로 했다. 5G 조기 구축과

In [54]:
print(len(naver_news_title[0]))
print(len(naver_news_content[0]))

3
3


### 크롤링 결과 저장하기

In [55]:
# 보통은 텍스트 파일로 만드는데, 데이터가 적은 경우 용이한 pickle 사용
# pickle 쓰는 이유는 저장된 리스트 그자체를 저장해서 그자체를 이용하려고
with open("naver_news_title.pk", "wb") as f:
    pickle.dump(naver_news_title, f)
    
with open("naver_news_contecnt.pk", "wb") as f:
    pickle.dump(naver_news_content, f)

- 이제 크롤링한 것들을 형태소 분석기를 불러와서 konlpy 설치하고
- konlpy 불러와서 정제하고, 토픽모델링하는 것까지 하나하나 배워보자